In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ftx

In [ ]:
df = pd.read_csv('trades_1.csv')
df = df.iloc[::-1]
df.head(0)

In [ ]:
df_deposits = pd.read_csv('deposits.csv')
df_deposits = df_deposits.iloc[::-1]
df_deposits.head(0)

In [ ]:
df_withdrawals = pd.read_csv('withdrawals.csv')
df_withdrawals = df_withdrawals.iloc[::-1]
df_withdrawals.head(0)

In [ ]:
wallet = {}

for index, row in df.iterrows():
    # spot
    if '/' in row.Market:
        asset, currency = row.Market.split('/')

        if currency not in wallet.keys():
            wallet[currency] = 0
        if asset not in wallet.keys():
            wallet[asset] = 0

        if row.Side == 'buy':
            wallet[currency] -= row.Total
            wallet[asset] += row.Size
        elif row.Side == 'sell':
            wallet[currency] += row.Total
            wallet[asset] -= row.Size
        else:
            raise ValueError

        wallet[row['Fee Currency']] -= row.Fee
    # derivative
    elif '-' in row.Market:
        assert(row['Fee Currency'] == 'USD')
        
        if row.Market not in wallet.keys():
            wallet[row.Market] = 0
            
        if row.Side == 'buy':
            wallet['USD'] -= row.Total
            wallet[row.Market] += row.Size
        elif row.Side == 'sell':
            wallet['USD'] += row.Total
            wallet[row.Market] -= row.Size
        else:
            raise ValueError
            
        wallet[row['Fee Currency']] -= row.Fee
    else:
        print("ERROR:")
        print(row)
        raise NotImplementedError
        

In [ ]:
wallet_derivative = {'USD' : 0}

for index, row in df.iterrows():
    # derivative
    if '-' in row.Market:
        if row.Market not in wallet_derivative.keys():
            wallet_derivative[row.Market] = 0

        assert(row['Fee Currency'] == 'USD')
            
        if row.Side == 'buy':
            wallet_derivative['USD'] -= row.Total
            wallet_derivative[row.Market] += row.Size
        elif row.Side == 'sell':
            wallet_derivative['USD'] += row.Total
            wallet_derivative[row.Market] -= row.Size
        else:
            raise ValueError
            
        wallet_derivative[row['Fee Currency']] -= row.Fee


In [ ]:
wallet_fee = {}

for index, row in df.iterrows():
    # spot
    if '/' in row.Market:
        asset, currency = row.Market.split('/')

        if currency not in wallet_fee.keys():
            wallet_fee[currency] = 0
        if asset not in wallet_fee.keys():
            wallet_fee[asset] = 0

        wallet_fee[row['Fee Currency']] -= row.Fee
    # derivative
    elif '-' in row.Market:
        assert(row['Fee Currency'] == 'USD')
        
        if row.Market not in wallet_fee.keys():
            wallet_fee[row.Market] = 0
            
        wallet_fee[row['Fee Currency']] -= row.Fee
    else:
        print("ERROR:")
        print(row)
        raise NotImplementedError
        

In [ ]:
wallet_out = {}

for index, row in df_deposits.iterrows():
    if row.Coin not in wallet.keys():
        wallet[row.Coin] = 0
    if row.Coin not in wallet_out.keys():
        wallet_out[row.Coin] = 0
        
    wallet[row.Coin] += row.Amount
    wallet_out[row.Coin] -= row.Amount
    
    
for index, row in df_withdrawals.iterrows():
    if row.Coin not in wallet.keys():
        wallet[row.Coin] = 0
    if row.Coin not in wallet_out.keys():
        wallet_out[row.Coin] = 0
        
    wallet[row.Coin] -= row.Amount
    wallet_out[row.Coin] += row.Amount - row.fee

In [ ]:
epsilon = 1e-4
wallet = {key:val for key, val in wallet.items() if val > epsilon or val < -epsilon}
wallet_derivative = {key:val for key, val in wallet_derivative.items() if val > epsilon or val < -epsilon}
wallet_fee = {key:val for key, val in wallet_fee.items() if val > epsilon or val < -epsilon}
wallet_out = {key:val for key, val in wallet_out.items() if val > epsilon or val < -epsilon}

In [ ]:
wallet

In [ ]:
wallet_out

In [ ]:
wallet_derivative

In [ ]:
wallet_fee

In [ ]:
client = ftx.FtxClient()

# client.get_trades('BTC/USD', 2, datetime.datetime(2021, 1, 1).timestamp(), datetime.datetime(2021, 1, 2).timestamp())

def eval_wallet(w):
    total = 0
    for k in w.keys():
        if '-' in k:
            orderbook = client.get_orderbook(k, 1)
            value = np.mean(np.concatenate([np.array(orderbook['bids'])[:, 0], np.array(orderbook['asks'])[:, 0]]))
        elif k != 'USD':
            orderbook = client.get_orderbook(f'{k}/USD', 1)
            if orderbook['bids'] == [] or orderbook['asks'] == []:
                value = 0
            else:
                value = np.mean(np.concatenate([np.array(orderbook['bids'])[:, 0], np.array(orderbook['asks'])[:, 0]]))
        else:
            value = 1.0

        total += w[k] * value
    return total

In [ ]:
value_wallet = eval_wallet(wallet)
value_wallet_derivative = eval_wallet(wallet_derivative)
value_wallet_fee = eval_wallet(wallet_fee)
value_wallet_out = eval_wallet(wallet_out)

In [ ]:
results = value_wallet + value_wallet_out
print("wallet value".ljust(32), ':', round(value_wallet, 2))
print("wallet 'out' value".ljust(32), ':', round(value_wallet_out, 2))
print("results".ljust(32), ':', round(results, 2))
print("fee sum".ljust(32), ':', round(value_wallet_fee, 2))
print("derivative".ljust(32), ':', round((1-((results - value_wallet_derivative) / results))*100, 2), '%')